# Entrenamiento con Simple Transformers del model BERT

## Paso 1: Cargar datos en Google Colab

NameError: name 'requests' is not defined

## Paso 2: Instalar librerías

In [1]:
!pip install transformers evaluate torch --quiet
!pip install simpletransformers transformers datasets huggingface_hub scikit-learn
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.8/330.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 147.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=3ae863e877a4a4096cc706edae53cafd33b5031e228d26e2fce2aa4c0da8256b
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


## Paso 3: Cargar librerías

In [7]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import evaluate
import json
import os
import shutil
import requests


from tqdm import tqdm
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from sklearn.model_selection import train_test_split
from google.colab import files

In [3]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [4]:
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

GPU available: True
GPU name: NVIDIA A100-SXM4-80GB


## Paso 4: Cargar datos

In [14]:
with open("train_colombia_mexico_dataset.json", "r", encoding="utf-8") as f:
    train_dataset = json.load(f)

print(f"✅ Loaded {len(train_dataset)} records successfully!")

✅ Loaded 1 records successfully!


In [15]:
with open("eval_colombia_mexico_dataset.json", "r", encoding="utf-8") as f:
    eval_dataset = json.load(f)

print(f"✅ Loaded {len(eval_dataset)} records successfully!")

✅ Loaded 1 records successfully!


In [16]:
train_dataset = train_dataset["data"]

In [17]:
eval_dataset = eval_dataset["data"]

In [18]:
# 🔧 1️⃣ Flatten your dataset so each row has 'context' and 'qas'
def flatten_squad(dataset):
    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data

In [19]:
train_data = flatten_squad(train_dataset)

In [20]:
eval_data = flatten_squad(eval_dataset)

In [21]:
print(f"✅ Training samples: {len(train_data)}")
print(f"✅ Eval samples: {len(eval_data)}")

✅ Training samples: 3235
✅ Eval samples: 2845


## Paso 5: Definir hiperparámetros

In [22]:
model_args = QuestionAnsweringArgs()

# Training behavior
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 4
model_args.learning_rate = 2e-5
model_args.gradient_accumulation_steps = 1
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1
model_args.best_model_dir = "./outputs/best_model/"
model_args.output_dir = "./outputs/"

# Optimization
model_args.max_seq_length = 384
model_args.doc_stride = 128
model_args.warmup_ratio = 0.1
model_args.max_answer_length = 30

# Logging
model_args.logging_steps = 100
model_args.evaluate_during_training_verbose = True
model_args.manual_seed = 42

# Resource handling
model_args.use_multiprocessing = False  # safer for notebooks
model_args.fp16 = torch.cuda.is_available()  # use mixed precision if CUDA available

## Paso 6: Cargar el modelo

In [25]:
model_original = QuestionAnsweringModel(
    model_type="bert",
    model_name="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",  # BETO distilled
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

The following layers were not sharded: bert.encoder.layer.*.attention.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, qa_outputs.weight, bert.encoder.layer.*.attention.self.query.weight, qa_outputs.bias, bert.encoder.layer.*.attention.self.value.bias, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.self.key.bias, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.query.bias, bert.enco

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [26]:
model = QuestionAnsweringModel(
    model_type="bert",
    model_name="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",  # BETO
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

The following layers were not sharded: bert.encoder.layer.*.attention.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, qa_outputs.weight, bert.encoder.layer.*.attention.self.query.weight, qa_outputs.bias, bert.encoder.layer.*.attention.self.value.bias, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.self.key.bias, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.query.bias, bert.enco

## Paso 7: Entrenamiento del modelo

In [ ]:
model.train_model(train_data, eval_data=eval_data)

add example index and unique id: 100%|██████████| 14613/14613 [00:00<00:00, 653272.84it/s]


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:697: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 4:   0%|          | 0/8698 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:720: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


convert squad examples to features:   0%|          | 0/6263 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/6263 [00:26<46:14:32, 26.58s/it]

convert squad examples to features:  24%|██▍       | 1501/6263 [00:28<01:06, 71.57it/s]

convert squad examples to features:  72%|███████▏  | 4501/6263 [00:30<00:07, 245.35it/s]

convert squad examples to features:  80%|███████▉  | 5001/6263 [00:37<00:06, 183.04it/s]

convert squad examples to features: 100%|██████████| 6263/6263 [00:40<00:00, 155.97it/s]


add example index and unique id: 100%|██████████| 6263/6263 [00:00<00:00, 417796.04it/s]


...



convert squad examples to features:   0%|          | 0/6263 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/6263 [00:26<46:54:59, 26.97s/it]

convert squad examples to features:  24%|██▍       | 1501/6263 [00:29<01:08, 69.32it/s]

convert squad examples to features:  40%|███▉      | 2501/6263 [00:30<00:29, 127.75it/s]

convert squad examples to features:  80%|███████▉  | 5001/6263 [00:34<00:04, 254.38it/s]

convert squad examples to features: 100%|██████████| 6263/6263 [00:41<00:00, 151.38it/s]


add example index and unique id: 100%|██████████| 6263/6263 [00:00<00:00, 367551.78it/s]


Running Evaluation:   0%|          | 0/3775 [00:00<?, ?it/s]



convert squad examples to features:   0%|          | 0/6263 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/6263 [00:24<43:23:03, 24.94s/it]

convert squad examples to features:  32%|███▏      | 2001/6263 [00:26<00:39, 106.86it/s]

convert squad examples to features:  40%|███▉      | 2501/6263 [00:31<00:36, 102.78it/s]

convert squad examples to features:  80%|███████▉  | 5001/6263 [00:34<00:05, 248.29it/s]

convert squad examples to features: 100%|██████████| 6263/6263 [00:40<00:00, 154.54it/s]


add example index and unique id: 100%|██████████| 6263/6263 [00:00<00:00, 374799.19it/s]


Running Evaluation:   0%|          | 0/3775 [00:00<?, ?it/s]



convert squad examples to features:   0%|          | 0/6263 [00:00<?, ?it/s]

## Paso 8: Evaluación de los resultados

In [ ]:
result_original, texts_original = model_original.eval_model(eval_data)
print("📊 Evaluation results:")
print(result_original)

In [ ]:
result, texts = model.eval_model(eval_data)
print("📊 Evaluation results:")
print(result)

In [ ]:
correct = result['correct']
similar = result['similar']
incorrect = result['incorrect']
total = correct + similar + incorrect

# 1️⃣ Exact Match Accuracy
exact_match = correct / total

# 2️⃣ Weighted Accuracy (partial credit for 'similar')
weighted_accuracy = (correct + 0.5 * similar) / total

# 3️⃣ F1 Score approximation
TP = correct + 0.5 * similar
FN = 0.5 * similar + incorrect
# Assuming FP = 0 (as Simple Transformers counts predictions, not negatives)
precision = TP / TP
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print results
print(f"Exact Match (EM): {exact_match:.4f} → {exact_match*100:.2f}%")
print(f"Weighted Accuracy: {weighted_accuracy:.4f} → {weighted_accuracy*100:.2f}%")
print(f"F1 Score: {f1_score:.4f} → {f1_score*100:.2f}%")

## Paso 9: Guardar los resultados

In [ ]:
# Folder to save
local_path = "./QA_model_bert"
os.makedirs(local_path, exist_ok=True)

# Save the Hugging Face model & tokenizer directly
model.model.save_pretrained(local_path)       # Saves weights + config
model.tokenizer.save_pretrained(local_path)   # Saves vocab + tokenizer config

# Check files
!ls -l ./QA_model

In [ ]:
shutil.make_archive("QA_model_bert", 'zip', local_path)
print("✅ Zipped model")
!ls -lh QA_model.zip

In [ ]:
files.download("QA_model_bert.zip")

In [ ]:

shutil.move("QA_model_bert.zip", "/content/drive/MyDrive/Thesis_QA_Optimization/Model")

In [ ]:
# Path to the folder containing the saved model
model_path = "./QA_model"  # change if different

# Reload the model
my_model = QuestionAnsweringModel(
    "bert",
    model_path,
    use_cuda=True  # set to False if no GPU
)

In [ ]:
# Context & question
context = "Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez, fue hallado muerto en Tamaulipas."
question = "¿Quién fue hallado muerto en Tamaulipas?"

# Prepare input in SimpleTransformers format
to_predict = [
    {
        "context": context,
        "qas": [
            {
                "id": "0",
                "question": question,
                "answers": [{"text": " ", "answer_start": 0}],
                "is_impossible": False
            }
        ]
    }
]

# Run prediction
answers = my_model.predict(to_predict)
print(answers)